# 03_Regression.py

This notebook was automatically converted from a Python script.

In [ ]:

"""
Scikit-learn Regresyon Algoritmaları

Bu script, scikit-learn kütüphanesindeki temel regresyon algoritmalarını göstermektedir.
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_boston, load_diabetes, make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings



Boston housing dataset'i kaldırıldığı için eğer hata alınırsa


California Housing veri setini kullanabiliriz


In [ ]:
try:
    from sklearn.datasets import fetch_california_housing
    housing = fetch_california_housing()
    boston_available = False
except:
    housing = None
    boston_available = True



Regresyon modelleri


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR



Uyarıları gizle


In [ ]:
warnings.filterwarnings('ignore')

print("Scikit-learn Regresyon Algoritmaları")
print("="*50)



1. Veri Yükleme ve Ön İşleme


In [ ]:
print("\n1. Veri Yükleme ve Ön İşleme")
print("-"*50)



Veri seti yükleme


In [ ]:
if boston_available:
    try:
        boston = load_boston()
        X = boston.data
        y = boston.target
        feature_names = boston.feature_names
        dataset_name = "Boston Housing"
    except:
        # Eğer Boston dataset'i artık kullanılamıyorsa
        boston_available = False

if not boston_available:
    housing = fetch_california_housing()
    X = housing.data
    y = housing.target
    feature_names = housing.feature_names
    dataset_name = "California Housing"

print(f"{dataset_name} veri seti yüklendi.")
print("Veri seti boyutu:", X.shape)
print("Özellik isimleri:", feature_names)



Eğitim ve test kümelerine ayırma


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Veri ölçeklendirme


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



2. Basit Doğrusal Regresyon


In [ ]:
print("\n2. Basit Doğrusal Regresyon")
print("-"*50)



Doğrusal regresyon modeli


In [ ]:
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)



Tahmin


In [ ]:
y_pred_lr = lr.predict(X_test_scaled)



Performans metrikleri


In [ ]:
mse = mean_squared_error(y_test, y_pred_lr)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_lr)
mae = mean_absolute_error(y_test, y_pred_lr)

print("Doğrusal Regresyon Performansı:")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")
print(f"MAE: {mae:.4f}")



Katsayılar ve kesişim


In [ ]:
print("\nKesişim (Intercept):", lr.intercept_)
print("\nKatsayılar (Coefficients):")
coefficients = pd.DataFrame({
    'Özellik': feature_names,
    'Katsayı': lr.coef_
})
print(coefficients.sort_values(by='Katsayı', ascending=False))



Gerçek vs Tahmin grafiği


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_lr, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
plt.xlabel('Gerçek Değerler')
plt.ylabel('Tahmin Edilen Değerler')
plt.title('Doğrusal Regresyon: Gerçek vs Tahmin')
plt.grid(True)
plt.savefig('linear_regression_true_vs_pred.png')
plt.close()
print("Doğrusal regresyon gerçek vs tahmin grafiği 'linear_regression_true_vs_pred.png' olarak kaydedildi.")



3. Polinom Regresyon


In [ ]:
print("\n3. Polinom Regresyon")
print("-"*50)



Polinom regresyon için pipeline


In [ ]:
degrees = [1, 2, 3]
rmse_scores = []
r2_scores = []

for degree in degrees:
    polynomial_features = PolynomialFeatures(degree=degree, include_bias=False)
    linear_regression = LinearRegression()
    
    pipeline = Pipeline([
        ("polynomial_features", polynomial_features),
        ("linear_regression", linear_regression)
    ])
    
    pipeline.fit(X_train_scaled, y_train)
    y_pred = pipeline.predict(X_test_scaled)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    rmse_scores.append(rmse)
    r2_scores.append(r2)
    
    print(f"Polinom Derecesi {degree}:")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}\n")



Performans karşılaştırma grafiği


In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(121)
plt.plot(degrees, rmse_scores, marker='o', linestyle='-')
plt.xlabel('Polinom Derecesi')
plt.ylabel('RMSE')
plt.title('Polinom Derecesi vs RMSE')
plt.grid(True)

plt.subplot(122)
plt.plot(degrees, r2_scores, marker='o', linestyle='-', color='orange')
plt.xlabel('Polinom Derecesi')
plt.ylabel('R²')
plt.title('Polinom Derecesi vs R²')
plt.grid(True)

plt.tight_layout()
plt.savefig('polynomial_regression_comparison.png')
plt.close()
print("Polinom regresyon karşılaştırma grafiği 'polynomial_regression_comparison.png' olarak kaydedildi.")



4. Düzenleştirilmiş (Regularized) Regresyon


In [ ]:
print("\n4. Düzenleştirilmiş (Regularized) Regresyon")
print("-"*50)



Ridge Regresyon (L2 düzenleştirme)


In [ ]:
ridge = Ridge(alpha=1.0)
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_scaled)
ridge_rmse = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
ridge_r2 = r2_score(y_test, y_pred_ridge)



Lasso Regresyon (L1 düzenleştirme)


In [ ]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)
y_pred_lasso = lasso.predict(X_test_scaled)
lasso_rmse = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
lasso_r2 = r2_score(y_test, y_pred_lasso)



ElasticNet (L1 ve L2 düzenleştirme kombinasyonu)


In [ ]:
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic.fit(X_train_scaled, y_train)
y_pred_elastic = elastic.predict(X_test_scaled)
elastic_rmse = np.sqrt(mean_squared_error(y_test, y_pred_elastic))
elastic_r2 = r2_score(y_test, y_pred_elastic)

print("Ridge Regresyon (L2):")
print(f"RMSE: {ridge_rmse:.4f}")
print(f"R²: {ridge_r2:.4f}\n")

print("Lasso Regresyon (L1):")
print(f"RMSE: {lasso_rmse:.4f}")
print(f"R²: {lasso_r2:.4f}\n")

print("ElasticNet (L1 + L2):")
print(f"RMSE: {elastic_rmse:.4f}")
print(f"R²: {elastic_r2:.4f}\n")



Katsayı karşılaştırma


In [ ]:
coef_df = pd.DataFrame({
    'Özellik': feature_names,
    'Doğrusal': lr.coef_,
    'Ridge': ridge.coef_,
    'Lasso': lasso.coef_,
    'ElasticNet': elastic.coef_
})



Katsayıların görselleştirilmesi


In [ ]:
plt.figure(figsize=(14, 8))
bar_width = 0.2
index = np.arange(len(feature_names))

plt.bar(index, coef_df['Doğrusal'], bar_width, label='Doğrusal')
plt.bar(index + bar_width, coef_df['Ridge'], bar_width, label='Ridge')
plt.bar(index + 2*bar_width, coef_df['Lasso'], bar_width, label='Lasso')
plt.bar(index + 3*bar_width, coef_df['ElasticNet'], bar_width, label='ElasticNet')

plt.xlabel('Özellikler')
plt.ylabel('Katsayı Değeri')
plt.title('Regresyon Modellerinin Katsayı Karşılaştırması')
plt.xticks(index + 1.5*bar_width, feature_names, rotation=90)
plt.legend()
plt.tight_layout()
plt.savefig('regularized_regression_coefficients.png')
plt.close()
print("Düzenleştirilmiş regresyon katsayı karşılaştırma grafiği 'regularized_regression_coefficients.png' olarak kaydedildi.")



5. Karar Ağacı Regresyonu


In [ ]:
print("\n5. Karar Ağacı Regresyonu")
print("-"*50)



Karar Ağacı


In [ ]:
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train, y_train)  # Ölçeklendirmeye gerek yok
y_pred_dt = dt_reg.predict(X_test)
dt_rmse = np.sqrt(mean_squared_error(y_test, y_pred_dt))
dt_r2 = r2_score(y_test, y_pred_dt)

print("Karar Ağacı Regresyonu:")
print(f"RMSE: {dt_rmse:.4f}")
print(f"R²: {dt_r2:.4f}\n")



Özellik önem dereceleri


In [ ]:
feature_importance = dt_reg.feature_importances_
indices = np.argsort(feature_importance)

plt.figure(figsize=(10, 6))
plt.barh(range(len(indices)), feature_importance[indices], color='skyblue')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Özellik Önem Derecesi')
plt.title('Karar Ağacı - Özellik Önem Dereceleri')
plt.tight_layout()
plt.savefig('decision_tree_regression_feature_importance.png')
plt.close()
print("Karar ağacı regresyonu özellik önem dereceleri grafiği 'decision_tree_regression_feature_importance.png' olarak kaydedildi.")



6. Rastgele Orman Regresyonu


In [ ]:
print("\n6. Rastgele Orman Regresyonu")
print("-"*50)



Rastgele Orman


In [ ]:
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg.fit(X_train, y_train)  # Ölçeklendirmeye gerek yok
y_pred_rf = rf_reg.predict(X_test)
rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_rf))
rf_r2 = r2_score(y_test, y_pred_rf)

print("Rastgele Orman Regresyonu:")
print(f"RMSE: {rf_rmse:.4f}")
print(f"R²: {rf_r2:.4f}\n")



Özellik önem dereceleri


In [ ]:
feature_importance = rf_reg.feature_importances_
indices = np.argsort(feature_importance)

plt.figure(figsize=(10, 6))
plt.barh(range(len(indices)), feature_importance[indices], color='lightgreen')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Özellik Önem Derecesi')
plt.title('Rastgele Orman - Özellik Önem Dereceleri')
plt.tight_layout()
plt.savefig('random_forest_regression_feature_importance.png')
plt.close()
print("Rastgele orman regresyonu özellik önem dereceleri grafiği 'random_forest_regression_feature_importance.png' olarak kaydedildi.")



7. Destek Vektör Regresyonu (SVR)


In [ ]:
print("\n7. Destek Vektör Regresyonu (SVR)")
print("-"*50)



SVR


In [ ]:
svr = SVR(kernel='rbf')
svr.fit(X_train_scaled, y_train)
y_pred_svr = svr.predict(X_test_scaled)
svr_rmse = np.sqrt(mean_squared_error(y_test, y_pred_svr))
svr_r2 = r2_score(y_test, y_pred_svr)

print("Destek Vektör Regresyonu (SVR):")
print(f"RMSE: {svr_rmse:.4f}")
print(f"R²: {svr_r2:.4f}\n")



Farklı kernel fonksiyonları


In [ ]:
kernels = ['linear', 'poly', 'rbf']
svr_models = {}
svr_rmse_scores = []
svr_r2_scores = []

for kernel in kernels:
    svr = SVR(kernel=kernel)
    svr.fit(X_train_scaled, y_train)
    y_pred = svr.predict(X_test_scaled)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    svr_rmse_scores.append(rmse)
    svr_r2_scores.append(r2)
    
    print(f"SVR ({kernel} kernel):")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}\n")



Kernel karşılaştırma grafiği


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(121)
plt.bar(kernels, svr_rmse_scores, color='salmon')
plt.xlabel('Kernel Tipi')
plt.ylabel('RMSE')
plt.title('SVR - Kernel vs RMSE')
plt.tick_params(axis='x', rotation=45)

plt.subplot(122)
plt.bar(kernels, svr_r2_scores, color='skyblue')
plt.xlabel('Kernel Tipi')
plt.ylabel('R²')
plt.title('SVR - Kernel vs R²')
plt.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('svr_kernel_comparison.png')
plt.close()
print("SVR kernel karşılaştırma grafiği 'svr_kernel_comparison.png' olarak kaydedildi.")



8. Gradyan Artırma (Gradient Boosting)


In [ ]:
print("\n8. Gradyan Artırma (Gradient Boosting)")
print("-"*50)



Gradient Boosting Regresyonu


In [ ]:
gb_reg = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_reg.fit(X_train, y_train)
y_pred_gb = gb_reg.predict(X_test)
gb_rmse = np.sqrt(mean_squared_error(y_test, y_pred_gb))
gb_r2 = r2_score(y_test, y_pred_gb)

print("Gradient Boosting Regresyonu:")
print(f"RMSE: {gb_rmse:.4f}")
print(f"R²: {gb_r2:.4f}\n")



Özellik önem dereceleri


In [ ]:
feature_importance = gb_reg.feature_importances_
indices = np.argsort(feature_importance)

plt.figure(figsize=(10, 6))
plt.barh(range(len(indices)), feature_importance[indices], color='lightcoral')
plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
plt.xlabel('Özellik Önem Derecesi')
plt.title('Gradient Boosting - Özellik Önem Dereceleri')
plt.tight_layout()
plt.savefig('gradient_boosting_feature_importance.png')
plt.close()
print("Gradient Boosting regresyonu özellik önem dereceleri grafiği 'gradient_boosting_feature_importance.png' olarak kaydedildi.")



9. Model Karşılaştırma


In [ ]:
print("\n9. Model Karşılaştırma")
print("-"*50)



Tüm modelleri karşılaştırma


In [ ]:
models = {
    'Doğrusal Regresyon': (lr, X_train_scaled, X_test_scaled, True),
    'Ridge': (ridge, X_train_scaled, X_test_scaled, True),
    'Lasso': (lasso, X_train_scaled, X_test_scaled, True),
    'ElasticNet': (elastic, X_train_scaled, X_test_scaled, True),
    'Karar Ağacı': (dt_reg, X_train, X_test, False),
    'Rastgele Orman': (rf_reg, X_train, X_test, False),
    'SVR (rbf)': (SVR(kernel='rbf'), X_train_scaled, X_test_scaled, True),
    'Gradient Boosting': (gb_reg, X_train, X_test, False)
}

results = {}

for name, (model, X_tr, X_te, _) in models.items():
    if name not in ['Karar Ağacı', 'Rastgele Orman', 'Gradient Boosting']:
        model.fit(X_tr, y_train)
    
    y_pred = model.predict(X_te)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    results[name] = {'RMSE': rmse, 'R²': r2, 'MAE': mae}



Sonuçları DataFrame olarak gösterme


In [ ]:
results_df = pd.DataFrame(results).T
print(results_df.sort_values(by='RMSE'))



Performans karşılaştırma grafikleri


In [ ]:
plt.figure(figsize=(16, 6))

plt.subplot(121)
results_df.sort_values(by='RMSE')['RMSE'].plot(kind='bar', ax=plt.gca(), color='salmon')
plt.title('RMSE Karşılaştırması (Düşük Daha İyi)')
plt.ylabel('RMSE')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.subplot(122)
results_df.sort_values(by='R²', ascending=False)['R²'].plot(kind='bar', ax=plt.gca(), color='skyblue')
plt.title('R² Karşılaştırması (Yüksek Daha İyi)')
plt.ylabel('R²')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.savefig('regression_models_comparison.png')
plt.close()
print("Regresyon modelleri karşılaştırma grafiği 'regression_models_comparison.png' olarak kaydedildi.") 
